In [36]:
import pandas as pd

# Sample training data
train_data = {
    'Category': ['A', 'B', 'C', 'A', 'C'],
    'Type': ['X', 'Y', 'X', 'Y', 'X']
}
train_df = pd.DataFrame(train_data)

# Sample test data (in original categorical form)
test_data = {
    'Category': ['B', 'A'],
    'Type': ['Y', 'X']
}
test_df = pd.DataFrame(test_data)

# Apply get_dummies on the training data
train_dummies = pd.get_dummies(train_df, drop_first=False)

# Apply get_dummies on the test data (without drop_first, same as train)
test_dummies = pd.get_dummies(test_df, drop_first=False)
display(test_dummies)
# Align the test data with the training data
# Reindex the test data columns to match the training data columns
test_dummies = test_dummies.reindex(columns=train_dummies.columns, fill_value=0)
display(test_dummies)
# Now, test_dummies has the same columns as train_dummies (including missing columns set to 0)
print("Training Dummies:")
print(train_dummies)

print("\nTest Dummies (aligned with train data):")
print(test_dummies)



,Category_A,Category_B,Type_X,Type_Y
0,False,True,False,True
1,True,False,True,False


,Category_A,Category_B,Category_C,Type_X,Type_Y
0,False,True,0,False,True
1,True,False,0,True,False


Training Dummies:
   Category_A  Category_B  Category_C  Type_X  Type_Y
0        True       False       False    True   False
1       False        True       False   False    True
2       False       False        True    True   False
3        True       False       False   False    True
4       False       False        True    True   False

Test Dummies (aligned with train data):
   Category_A  Category_B  Category_C  Type_X  Type_Y
0       False        True           0   False    True
1        True       False           0    True   False


In [3]:
import pandas as pd
import os
curr_dir = os.getcwd()
Data = pd.read_csv(os.path.join(curr_dir, os.pardir, 'data', 'raw_data.csv'))
display(Data.info())
display(Data.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   Department             14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


None

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000
mean,0.612834,0.716102,3.803054,201.050337,3.498233,0.144610,0.238083,0.021268
std,0.248631,0.171169,1.232592,49.943099,1.460136,0.351719,0.425924,0.144281
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [14]:
D = dict()
for col, t in dict(Data.dtypes).items():
    D[col] = dict()
    D[col]['label'] = col.capitalize().replace('_', ' ')
    if Data[col].nunique()<=10:
        D[col]['type'] = "dropdown"
        D[col]['options'] = sorted(Data[col].unique().tolist())
    else:
        D[col]['type'] = "value"
        D[col]['range'] = dict({'min': int(Data[col].min()), 'max': int(Data[col].max())})
        if t=='float':
            D[col]['precision'] = 10**(-int(Data[col].map(lambda x: len(str(x).split('.')[1])).max()))

In [16]:
D

{'satisfaction_level': {'label': 'Satisfaction level',
  'type': 'value',
  'range': {'min': 0, 'max': 1},
  'precision': 0.01},
 'last_evaluation': {'label': 'Last evaluation',
  'type': 'value',
  'range': {'min': 0, 'max': 1},
  'precision': 0.01},
 'number_project': {'label': 'Number project',
  'type': 'dropdown',
  'options': [2, 3, 4, 5, 6, 7]},
 'average_montly_hours': {'label': 'Average montly hours',
  'type': 'value',
  'range': {'min': 96, 'max': 310}},
 'time_spend_company': {'label': 'Time spend company',
  'type': 'dropdown',
  'options': [2, 3, 4, 5, 6, 7, 8, 10]},
 'Work_accident': {'label': 'Work accident',
  'type': 'dropdown',
  'options': [0, 1]},
 'left': {'label': 'Left', 'type': 'dropdown', 'options': [0, 1]},
 'promotion_last_5years': {'label': 'Promotion last 5years',
  'type': 'dropdown',
  'options': [0, 1]},
 'Department': {'label': 'Department',
  'type': 'dropdown',
  'options': ['IT',
   'RandD',
   'accounting',
   'hr',
   'management',
   'marketing',

In [116]:
D = dict([(col,dict({'label': col.capitalize().replace('_', ' '),
                 'type': 'dropdown' if Data[col].nunique()<=10 else 'range',
                 'options': sorted(Data[col].unique().tolist()) if Data[col].nunique()<=10  else [],
                 'range': None if Data[col].nunique()<=10  else dict({'min': int(Data[col].min()), 'max': int(Data[col].max())}),
                 'precision': int(Data[col].map(lambda x: len(str(x).split('.')[1])).max()) if t=='float' else None
                })) for col, t in dict(Data.dtypes).items()])

In [ ]:
import pandas as pd

data = pd.read_json('../data/onehot_name_dictionary.json', typ='series')

display(data)

display(data.value_counts()[data.value_counts()>1].index)
display(data.value_counts()[data.value_counts()==1].index)


print([col for col in set(data.values) if data[data==col].size==1])
print([col for col in set(data.values) if data[data==col].size>1])

satisfaction_level           satisfaction_level
last_evaluation                 last_evaluation
number_of_projects           number_of_projects
average_montly_hours       average_montly_hours
time_spend_in_company     time_spend_in_company
had_work_accidents           had_work_accidents
left_or_not                         left_or_not
promoted_last_5_years     promoted_last_5_years
department_IT                        department
department_RandD                     department
department_accounting                department
department_hr                        department
department_management                department
department_marketing                 department
department_product_mng               department
department_sales                     department
department_support                   department
department_technical                 department
salary_high                              salary
salary_low                               salary
salary_medium                           

Index(['department', 'salary'], dtype='object')

Index(['last_evaluation', 'satisfaction_level', 'number_of_projects',
       'average_montly_hours', 'had_work_accidents', 'time_spend_in_company',
       'promoted_last_5_years', 'left_or_not'],
      dtype='object')

['left_or_not', 'average_montly_hours', 'had_work_accidents', 'number_of_projects', 'promoted_last_5_years', 'last_evaluation', 'satisfaction_level', 'time_spend_in_company']
['salary', 'department']


In [5]:
!pip install --upgrade huggingface_hub

In [14]:
import os
from dotenv import load_dotenv
from huggingface_hub import HfApi, Repository


load_dotenv(override=True)
HF_GITHUB_ACTIONS_TOKEN = os.getenv("HF_GITHUB_ACTIONS_TOKEN")
api = HfApi()
api.token(HF_GITHUB_ACTIONS_TOKEN)


TypeError: 'NoneType' object is not callable

In [19]:
from huggingface_hub import login
login(HF_GITHUB_ACTIONS_TOKEN)

In [17]:
[s for s in dir(api) if ('token' in s)]

['get_token_permission', 'token']